In [5]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../src')
import geopandas as gpd
from ipyleaflet import Map, GeoData, LayersControl, WidgetControl
from ipywidgets import IntSlider, VBox, HTML, Button, HBox
import ipyleaflet as ipyl
import shapely
import ee

from gee import initialize_ee_with_credentials, get_s2_hsv_median, get_ee_image_url, get_planet_rgb_median, get_planet_ndvi_median, get_planet_hsv_median
from ui import BASEMAP_TILES

initialize_ee_with_credentials()


v2_det_5 = gpd.GeoDataFrame(
    geometry=gpd.read_parquet("gs://demeter-labs/coffee/detections/tile_classifier_predictions_v0_costa-rica_posw1.0_prob_0.5_postprocess.parquet").geometry)
v2_det_5 = gpd.GeoDataFrame(
    geometry=[v2_det_5.buffer(0.0001).union_all()], crs=v2_det_5.crs).explode(index_parts=True)


BOUNDARY_PATH = "/home/christopher.x.ren/earth-index-ml/places/costa_rica.geojson"
BOUNDARY = gpd.read_file(BOUNDARY_PATH)
boundary_geom = ee.Geometry(shapely.geometry.mapping(BOUNDARY.geometry.iloc[0]))

hsv_median = get_s2_hsv_median(boundary_geom, '2023-01-01', '2024-12-31')
planet_rgb = get_planet_rgb_median(boundary_geom, '2023-01-01', '2024-12-31')
planet_ndvi = get_planet_ndvi_median(boundary_geom, '2023-01-01', '2024-12-31')
planet_hsv = get_planet_hsv_median(boundary_geom, '2023-01-01', '2024-12-31')

# Get tile URLs
hsv_url = get_ee_image_url(hsv_median, {
    'min': [0, 0, 0],
    'max': [1, 1, 1],
    'bands': ['hue', 'saturation', 'value']
})
rgb_url = get_ee_image_url(planet_rgb, {
    'min': [0, 0, 0],
    'max': [2000, 2000, 2000],
    'bands': ['R', 'G', 'B']
})
ndvi_url = get_ee_image_url(planet_ndvi, {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
})
planet_hsv_url = get_ee_image_url(planet_hsv, {
    'min': [0, 0, 0],
    'max': [1, 1, 1],
    'bands': ['hue', 'saturation', 'value']
})

BASEMAP_TILES['HSV_MEDIAN'] = hsv_url
BASEMAP_TILES['PLANET_RGB'] = rgb_url
BASEMAP_TILES['PLANET_NDVI'] = ndvi_url
BASEMAP_TILES['PLANET_HSV'] = planet_hsv_url

current_basemap = 'GOOGLE_HYBRID'
basemap_layer = ipyl.TileLayer(url=BASEMAP_TILES[current_basemap], no_wrap=True, name='basemap')
m = Map(basemap=basemap_layer, center=(-2, 120), zoom=5, layout={'height':'600px'}, scroll_wheel_zoom=True)

v2_det_layer = GeoData(
    geo_dataframe=v2_det_5,
    style={'color': '#377eb8', 'fillOpacity': 0.1},
    name='V2 Detections'
)
m.add_layer(v2_det_layer)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_9319/3688194975.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometry=[v2_det_5.buffer(0.0001).union_all()], crs=v2_det_5.crs).explode(index_parts=True)


In [6]:
m

Map(center=[-2, 120], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [8]:
detections = gpd.read_parquet("gs://demeter-labs/coffee/detections/tile_classifier_predictions_v0_costa-rica_posw1.0_prob_0.5_postprocess.parquet")
detections.to_file("coffee_costa_rica_detections_0.5.shp")

/home/christopher.x.ren/miniforge3/envs/ei-nb-v2/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value 180385182.204809487 of field area of feature 3459 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
/home/christopher.x.ren/miniforge3/envs/ei-nb-v2/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value 101854172.729521349 of field area of feature 3677 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(


In [12]:
import geemap
import ee

import sys
sys.path.insert(0, '../src')
from gee import initialize_ee_with_credentials
initialize_ee_with_credentials()

fc = ee.FeatureCollection("projects/earthindex/assets/demeter/coffee_costarica_05_v0")

dw = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate('2023-01-01', '2023-12-31')
most_common = dw.select('label').reduce(ee.Reducer.mode())

canopy_height = ee.ImageCollection(
    "projects/meta-forest-monitoring-okw37/assets/CanopyHeight").mosaic()




eth_canopy_height = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1")

rasterized = fc.reduceToImage(
    properties=['area'],
    reducer=ee.Reducer.first()
)

most_common = most_common.updateMask(rasterized.gt(0))
canopy_height = canopy_height.updateMask(rasterized.gt(0))
eth_canopy_height = eth_canopy_height.updateMask(rasterized.gt(0))

m = geemap.Map(center=[9.5, -84.0], zoom=9)

m.addLayer(rasterized, {
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
}, 'Coffee Detections')

m.addLayer(most_common, {
    'min': 0,
    'max': 8,
    'palette': ['#419bdf', '#397d49', '#88b053', '#7a87c6', '#e49635', '#dfc35a', '#c4281b', '#a59b8f', '#b39fe1']
}, 'Most Common')

m.addLayer(canopy_height, {
    'min': 0,
    'max': 50,
    'palette': "viridis"
}, 'Canopy Height [meters]')

m.addLayer(eth_canopy_height, {
    'min': 0,
    'max': 50,
    'palette': "viridis"
}, 'ETH Canopy Height [meters]')

m




Map(center=[9.5, -84.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…